In [1]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv
from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:

import random
from datasets import load_dataset
import json


def batch_glaive_calls(batch: dict[str,list]) -> any:
    tools = []
    messages = []
    batch = [dict(zip(batch,t)) for t in zip(*batch.values())]
    
    for b in batch:
        new_tools = json.loads(b["tools"])
        print(new_tools)
        if not new_tools:
            continue
        
        # skip duplicate tool names
        skip = False
        for tool in new_tools:
            if tool['name'] in [t['name'] for t in tools]:
                skip = True
        if skip:
            continue
        
        tools += new_tools
        
        messages += json.loads(b["messages"])
        
    random.shuffle(tools)
    
    results = [{
        "messages": messages,
        "tools": tools,
    }]
    
    empty = {"messages": [], "tools": []}
    return {k: [stringify(dic[k]) if k in dic else 'null' for dic in results] for k in empty} if len(results) else empty


glaive_function_calling = load_dataset("lucaelin/glaive-function-calling-v2", split="train").filter(lambda x: json.loads(x["tools"]))

glaive_function_calling_batched = glaive_function_calling.map(
    batch_glaive_calls, 
    batched=True, batch_size=30, remove_columns=glaive_function_calling.column_names)

glaive_function_calling_batched.push_to_hub("lucaelin/glaive-function-calling-mutliturn-v2")

Map:   0%|          | 0/78362 [00:00<?, ? examples/s]

[{'name': 'get_exchange_rate', 'description': 'Get the exchange rate between two currencies', 'parameters': {'type': 'object', 'properties': {'base_currency': {'type': 'string', 'description': 'The currency to convert from'}, 'target_currency': {'type': 'string', 'description': 'The currency to convert to'}}, 'required': ['base_currency', 'target_currency']}}]
[{'name': 'get_news_headlines', 'description': 'Get the latest news headlines', 'parameters': {'type': 'object', 'properties': {'country': {'type': 'string', 'description': 'The country for which to fetch news'}}, 'required': ['country']}}]


TypeError: string indices must be integers, not 'str'